In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm_notebook
import sys
import os
import numpy as np

sys.path.append('/home/antixk/Anand/')
print(sys.path)

from NeuralBlocks.models.densenet import DenseNet

['/home/antixk/Anand/NeuralBlocks/notebooks', '/home/antixk/miniconda3/envs/main/lib/python37.zip', '/home/antixk/miniconda3/envs/main/lib/python3.7', '/home/antixk/miniconda3/envs/main/lib/python3.7/lib-dynload', '', '/home/antixk/miniconda3/envs/main/lib/python3.7/site-packages', '/home/antixk/miniconda3/envs/main/lib/python3.7/site-packages/IPython/extensions', '/home/antixk/.ipython', '/home/antixk/Anand/']


In [2]:
torch.manual_seed(2456)
cudnn.deterministic = True
cudnn.benchmark = False
np.random.seed(2456)

NUM_EPOCH = 200
BATCH_SIZE = 128
CHECKPOINT_INTERVAL = 100
LR = 0.0002
NORM = 'BN'
DATA_PATH = "/home/antixk/Anand/NeuralBlocks/data_utils/datasets/CIFAR10/"
SAVE_PATH = "/home/antixk/Anand/NeuralBlocks/experiments/CIFAR10/"

os.makedirs(SAVE_PATH, exist_ok=True)

In [3]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root=DATA_PATH, train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)

testset = torchvision.datasets.CIFAR10(root=DATA_PATH, train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        train_loss_log.append(train_loss/(batch_idx+1))
        train_acc_log.append( 100.*correct/total)
        
        if(batch_idx%CHECKPOINT_INTERVAL==0):
             print("Train Epoch [{:3d}/{:3d}]Batch [{:3d}/{:3d}] Loss: {:.3f} Acc {:.3f}%".format(epoch, NUM_EPOCH,batch_idx, len(trainloader),
                train_loss/(batch_idx+1), 100.*correct/total))

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            test_loss_log.append(test_loss/(batch_idx+1))
            test_acc_log.append( 100.*correct/total)
        
            if(batch_idx%CHECKPOINT_INTERVAL==0):
                print("Test Epoch [{:3d}/{:3d}]Batch [{:3d}/{:3d}] Loss: {:.3f} Acc {:.3f}%".format(epoch, NUM_EPOCH,batch_idx, len(testloader),
                test_loss/(batch_idx+1), 100.*correct/total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir(SAVE_PATH+'checkpoint'):
            os.mkdir(SAVE_PATH+'checkpoint')
        torch.save(state, SAVE_PATH+'checkpoint/ckpt.pth')
        best_acc = acc

In [5]:
best_acc = 0 

for lr in tqdm_notebook([0.0001, 0.001, 0.1]):
    for norm in tqdm_notebook([None,'BN', 'MSN', 'MSNTReLU']):
        net = DenseNet(3, 10, norm='MSNTReLU').cuda()
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(net.parameters(), lr=0.01)

        train_loss_log =[]
        train_acc_log = []
        test_loss_log =[]
        test_acc_log =[]

        for epoch in tqdm_notebook(range(NUM_EPOCH)):
            train(epoch)
            test(epoch)

        np.savetxt(SAVE_PATH+"DenseNet_Train_loss_{}_{}.npy".format(norm,lr), train_loss_log)  
        np.savetxt(SAVE_PATH+"DenseNet_Test_loss_{}_{}.npy".format(norm,lr), test_loss_log)    
        np.savetxt(SAVE_PATH+"DenseNet_Train_Acc_{}_{}.npy".format(norm,lr), train_acc_log)    
        np.savetxt(SAVE_PATH+"DenseNet_Test_Acc_{}_{}.npy".format(norm,lr), test_acc_log)   
        del net

Train Epoch [  0/200]Batch [  0/391] Loss: 2.309 Acc 10.156%
Train Epoch [  0/200]Batch [100/391] Loss: 2.259 Acc 26.400%
Train Epoch [  0/200]Batch [200/391] Loss: 1.998 Acc 32.093%
Train Epoch [  0/200]Batch [300/391] Loss: 1.886 Acc 34.923%
Test Epoch [  0/200]Batch [  0/ 79] Loss: 1.470 Acc 49.219%
Saving..
Train Epoch [  1/200]Batch [  0/391] Loss: 1.669 Acc 41.406%



KeyboardInterrupt: 

In [ ]:
print('ads{}'.format(None))